In [1]:
from tqdm import tqdm
from glob import glob
import json

In [2]:
malay_news = glob('/home/ubuntu/server2/google-translate-malay-news/*.requested')
len(malay_news)

22

In [3]:
data = []
for f in malay_news:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            data.append({
                'ms': l['src'],
                'en': l['r']['result']
            })

87609it [00:01, 86432.93it/s]
100000it [00:00, 191365.63it/s]
100000it [00:00, 194809.45it/s]
100000it [00:00, 195636.33it/s]
99897it [00:00, 201777.76it/s]
99999it [00:00, 195592.86it/s]
99992it [00:00, 202193.02it/s]
99994it [00:00, 264291.33it/s]
99997it [00:00, 182244.72it/s]
99998it [00:00, 202442.12it/s]
100000it [00:00, 191828.44it/s]
99989it [00:00, 195711.66it/s]
100000it [00:00, 216011.50it/s]
99994it [00:00, 210278.36it/s]
100000it [00:00, 195643.45it/s]
100000it [00:00, 191406.41it/s]
100000it [00:00, 201517.47it/s]
100000it [00:00, 209701.25it/s]
100000it [00:00, 218225.79it/s]
100000it [00:00, 195501.64it/s]
100000it [00:00, 206116.61it/s]
99995it [00:00, 232516.95it/s]


In [4]:
len(data)

2187464

In [5]:
data[1]

{'ms': 'Sebulan lagi sebelum tutup tirai 2020, ini berita tumpuan sepanjang November',
 'en': 'One more month before the curtain closes on 2020, this is the focus news throughout November'}

In [6]:
# !pip3 install transformers -U
from transformers import AutoTokenizer, AutoModel
import torch

In [7]:
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-en')

In [8]:
model = AutoModel.from_pretrained('BAAI/bge-large-en')
_ = model.cuda()

2023-09-17 19:36:32.088890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-17 19:36:32.317935: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-17 19:36:33.236373: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-09-17 19:36:33.236505: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [9]:
def embedding(encoded_input):
    encoded_input = {'input_ids': encoded_input.unsqueeze(0)}
    for k in encoded_input:
        encoded_input[k] = encoded_input[k].cuda()

    model_output = model(**encoded_input)
    sentence_embeddings = model_output[0][:, 0]
    v = sentence_embeddings[0].detach().cpu().numpy()
    return v

In [10]:
import numpy as np
from itertools import islice

def batched(iterable, n):
    """
    Batch data into tuples of length n. The last batch may be shorter.
    """
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := tuple(islice(it, n))):
        yield batch

def get_tokens(text):
    encoded_input = tokenizer.encode(text,
                             return_tensors='pt')
    return encoded_input[0]

def chunked_tokens(text, chunk_length):
    tokens = get_tokens(text)
    chunks_iterator = batched(tokens, chunk_length)
    yield from chunks_iterator
    
def len_safe_get_embedding(
    text,
    max_tokens=512,
    average=True
):
    chunk_embeddings = []
    chunk_lens = []
    for chunk in chunked_tokens(text, chunk_length=max_tokens):
        
        chunk = torch.Tensor(chunk).long()
        
        chunk_embeddings.append(embedding(chunk))
        chunk_lens.append(len(chunk))

    if average:
        chunk_embeddings = np.average(chunk_embeddings, axis=0, weights=chunk_lens)
        chunk_embeddings = chunk_embeddings / np.linalg.norm(chunk_embeddings)
        chunk_embeddings = chunk_embeddings.tolist()
    return chunk_embeddings

len_safe_get_embedding(data[1]['en'], 2)

[-0.014365814985575426,
 0.011136879327906331,
 0.0025416722556333565,
 0.0176315403479563,
 -0.012204821477650282,
 -0.02563865915660413,
 -0.009833962018237342,
 0.017567550638878972,
 0.037987892334981364,
 0.011387327151476574,
 0.03556085768978685,
 0.00013234880360862427,
 0.017471197155000322,
 -0.021291631805869092,
 -0.039456247270001625,
 -0.012525502612344985,
 -0.01966490716766338,
 -0.026824440528733447,
 -0.025542371413070235,
 0.0030708462514157852,
 -0.01580195257632112,
 0.007310865376223332,
 -0.050839540885856224,
 -0.03622081158233293,
 -0.02743028360915345,
 0.02695542998466681,
 0.012465342030808034,
 0.003741139234451849,
 0.0681350742658206,
 0.05797019281906865,
 -0.036376687291088984,
 -0.03815014520206774,
 0.012894045254624906,
 -0.053241131158387994,
 -0.028723053192507227,
 -0.02029615374580509,
 0.01979426302776301,
 -0.046659943279130874,
 -0.022254966697883533,
 -0.045218015450930196,
 0.019110304552869942,
 -0.003769623160739404,
 0.02703561564928931,


In [11]:
!mkdir malay-news

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
mkdir: cannot create directory ‘malay-news’: File exists


In [12]:
instruction = 'Represent this sentence for searching relevant passages: '
examples = [instruction + data[i]['en'] for i in range(10)]

In [13]:
vs = []
for s in examples:
    vs.append(len_safe_get_embedding(s))

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(vs)[1]

array([0.72293885, 1.        , 0.83765245, 0.84295304, 0.7508758 ,
       0.85175435, 0.76592483, 0.71803362, 0.80411183, 0.73771153])

In [15]:
examples[1], examples[5]

('Represent this sentence for searching relevant passages: One more month before the curtain closes on 2020, this is the focus news throughout November',
 'Represent this sentence for searching relevant passages: Before we move on, why not see what is interesting throughout November that is a public focus.')

In [16]:
import os

for i in tqdm(range(len(data))):
    filename = os.path.join('malay-news', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    try:
        v = len_safe_get_embedding(instruction + data[i]['en'])
        d = {
            'text': data[i],
            'v': v,
        }
        with open(filename, 'w') as fopen:
            fopen.write(f'{json.dumps(d)}\n')
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 2187464/2187464 [1:09:25<00:00, 525.13it/s]


In [17]:
!du -hs malay-news

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
51G	malay-news
